## 智谱章节作业

##### 使用官方提供的示例，成功微调出广告数据集，要求使用 Lora 进行微调：  

##### 1. 你能看到 loss 的下降，并在最终回到 3.2 左右。  

In [1]:
import json
from typing import Union
from pathlib import Path


def _resolve_path(path: Union[str, Path]) -> Path:
    return Path(path).expanduser().resolve()


def _mkdir(dir_name: Union[str, Path]):
    dir_name = _resolve_path(dir_name)
    if not dir_name.is_dir():
        dir_name.mkdir(parents=True, exist_ok=False)


def convert_adgen(data_dir: Union[str, Path], save_dir: Union[str, Path]):
    def _convert(in_file: Path, out_file: Path):
        _mkdir(out_file.parent)
        with open(in_file, encoding='utf-8') as fin:
            with open(out_file, 'wt', encoding='utf-8') as fout:
                for line in fin:
                    dct = json.loads(line)
                    sample = {'conversations': [
                        {'role': 'user', 'content': dct['content']},
                        {'role': 'assistant', 'content': dct['summary']}
                        ]}
                    fout.write(json.dumps(sample, ensure_ascii=False) + '\n')
    
    data_dir = _resolve_path(data_dir)
    save_dir = _resolve_path(save_dir)

    train_file = data_dir / 'train.json'
    if train_file.is_file():
        out_file = save_dir / train_file.relative_to(data_dir)
        _convert(train_file, out_file)
    
    dev_file = data_dir / 'dev.json'
    if dev_file.is_file():
        out_file = save_dir / dev_file.relative_to(data_dir)
        _convert(dev_file, out_file)


convert_adgen('data/AdvertiseGen', 'data/AdvertiseGen_fix')

In [1]:
!CUDA_VISIBLE_DEVICES=0 NCCL_P2P_DISABLE="1" NCCL_IB_DISABLE="1" python finetune_demo/finetune_hf.py  data/AdvertiseGen_fix  /root/huggingface/hub/chatglm3-6b  finetune_demo/configs/lora.yaml yes

Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.
Loading checkpoint shards: 100%|██████████████████| 7/7 [00:02<00:00,  3.15it/s]
trainable params: 1,949,696 || all params: 6,245,533,696 || trainable%: 0.0312
--> Model

--> model has 1.949696M params

train_dataset: Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 114599
})
val_dataset: Dataset({
    features: ['input_ids', 'output_ids'],
    num_rows: 1070
})
test_dataset: Dataset({
    features: ['input_ids', 'output_ids'],
    num_rows: 1070
})
--> Sanity check
           '[gMASK]': 64790 -> -100
               'sop': 64792 -> -100
          '<|user|>': 64795 -> -100
                  '': 30910 -> -100
                '\n': 13 -> -100
                  '': 30910 -> -100
                '类型': 33467 -> -100
                 '#': 31010 -> -100
                 '裤': 56532 -> -100
                 '*': 3

#### train/loss

![train/loss](zhipu.train_loss.png)

##### 2. 你需要自己适配 inference.py 中的代码，并迁移到其他的推理框架中。例如，basic_demo 中没有读取微调模型后的 adapter 的内容，你需要参考 inference.py 的代码并进行修改，让其他 demo 能读入你的微调代码，将其部署到 basic_demo 下的 gradio_demo 中，并能够通过 webui 来进行调用。

选定loss较低的 output/checkoutpoint-2800 ，更新 basic_demo/web_demo_gradio.py  

```python
MODEL_PATH = os.environ.get('MODEL_PATH', '/root/projects/LLM-quickstart/homework/output/checkpoint-2800')
```

In [1]:
! python basic_demo/web_demo_gradio.py

Loading checkpoint shards: 100%|██████████████████| 7/7 [00:03<00:00,  1.80it/s]
Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.
Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


====conversation====
 [{'role': 'user', 'content': '类型#裙*版型#显瘦*材质#网纱*风格#性感*裙型#百褶*裙下摆#压褶*裙长#连衣裙*裙衣门襟#拉链*裙衣门襟#套头*裙款式#拼接*裙款式#拉链*裙款式#木耳边*裙款式#抽褶*裙款式#不规则'}]


====conversation====
 [{'role': 'user', 'content': '类型#裙*版型#显瘦*材质#网纱*风格#性感*裙型#百褶*裙下摆#压褶*裙长#连衣裙*裙衣门襟#拉链*裙衣门襟#套头*裙款式#拼接*裙款式#拉链*裙款式#木耳边*裙款式#抽褶*裙款式#不规则'}, {'role': 'assistant', 'content': '一件时尚的纯色系连衣裙，采用小碎花图案进行点缀。别致的木耳边装饰在领口和袖子处，搭配上透视网纱元素的设计，使得整体更加优雅浪漫，也更具女人味儿；修身的剪裁设计修身又显高挑，而肩部及腰部的开叉处理更是让这款连衣裙充满了性感的气息！同时，加上侧边的隐形口袋以及前幅的金属扣锁带来方便实用性的同时在视觉上更具有立体感。'}, {'role': 'user', 'content': '类型#裙*版型#显瘦*材质#网纱*风格#性感*裙型#百褶*裙下摆#压褶*裙长#连衣裙*裙衣门襟#拉链*裙衣门襟#套头*裙款式#拼接*裙款式#拉链*裙款式#木耳边*裙款式#抽褶*裙款式#不规则'}]
^C
Keyboard inter

#### infer_gradio
![infer_gradio](zhipu.infer_gradio.png)